# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

Загрузите и подготовьте данные.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.tree import DecisionTreeRegressor

import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
data = pd.read_csv("/datasets/autos.csv")
display(data.head())
data.info()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


Есть пропуски в столбцах `VehicleType`, `Gearbox`, `Model`, `FuelType`, `NotRepaired`. Могут быть вызваны, как сбоем программы, так и невнимательностью пользователей.

In [3]:
data['Gearbox'].unique()

array(['manual', 'auto', nan], dtype=object)

В столбце `Gearbox` очень мало вариантов, нет робота и вариатора например. Заменю пропуски на `other`.

In [4]:
data['NotRepaired'].unique()

array([nan, 'yes', 'no'], dtype=object)

В столбце `NotRepaired` пропуски могут означать, что ремонтов не было, заменю `NaN` на `no`.

В столбцах `VehicleType`, `Model`, `FuelType` есть вариант `other`, соответственно, все `NaN` заменю на `other`.

In [5]:
data = data.fillna({"VehicleType" : 'other',
                    "Model" : 'other',
                    "FuelType" : 'other',
                    "Gearbox" : 'other',
                    "NotRepaired" : 'no'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          354369 non-null object
RegistrationYear     354369 non-null int64
Gearbox              354369 non-null object
Power                354369 non-null int64
Model                354369 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             354369 non-null object
Brand                354369 non-null object
NotRepaired          354369 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


Изменю тип данных столбцов `DateCrawled`, `DateCreated`, `LastSeen`

In [6]:
data = data.astype({'DateCrawled' : 'datetime64[ns]',
                   'DateCreated' : 'datetime64[ns]',
                   'LastSeen' : 'datetime64[ns]'})

Дальше я очищаю таблицу от строк, в которых год регистрации 2016, но месяц регистрации больше, чем месяц скачивания анкеты из базы.

In [7]:
data.drop(
    data[(data['DateCrawled'].dt.year == data['RegistrationYear'])
         & (data['DateCrawled'].dt.month < data['RegistrationMonth'])].index, inplace=True)

In [8]:
data["NumberOfPictures"].unique()

array([0])

Видно, что столбец `NumberOfPictures` абсолютно пустой, поэтому можно его удалить.

In [9]:
data = data.drop(['NumberOfPictures'], axis=1)

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что ты нашел и удалил этот константный признак.
    
</div>

In [10]:
data['RegistrationMonth'].value_counts()

0     37352
3     34373
6     30747
4     28731
5     28446
7     26620
10    25495
12    23821
11    23677
9     23313
1     23219
8     22079
2     21267
Name: RegistrationMonth, dtype: int64

Очень странно выглядит месяц регистрации 0. Возможно, машина слишком старая, или документы утеряны. Ничем заменять не буду.

In [11]:
data['RegistrationYear'].sort_values().unique()

array([1000, 1001, 1039, 1111, 1200, 1234, 1253, 1255, 1300, 1400, 1500,
       1600, 1602, 1688, 1800, 1910, 1915, 1919, 1920, 1923, 1925, 1927,
       1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938,
       1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2066, 2200, 2222, 2290, 2500, 2800, 2900, 3000,
       3200, 3500, 3700, 3800, 4000, 4100, 4500, 4800, 5000, 5300, 5555,
       5600, 5900, 5911, 6000, 6500, 7000, 7100, 7500, 7800, 8000, 8200,
       8455, 8500, 8888, 9000, 9229, 9450, 9996, 99

Я зашёл на сайт auto.ru и у них минимальный год начинается с 1890, а максимальный у нас может быть 2016, т.к. данные собраны именно в 2016. Все остальные значения удалю.

In [12]:
data = data[(data['RegistrationYear'] >= 1890) & (data['RegistrationYear'] <= 2016)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334544 entries, 0 to 354368
Data columns (total 15 columns):
DateCrawled          334544 non-null datetime64[ns]
Price                334544 non-null int64
VehicleType          334544 non-null object
RegistrationYear     334544 non-null int64
Gearbox              334544 non-null object
Power                334544 non-null int64
Model                334544 non-null object
Kilometer            334544 non-null int64
RegistrationMonth    334544 non-null int64
FuelType             334544 non-null object
Brand                334544 non-null object
NotRepaired          334544 non-null object
DateCreated          334544 non-null datetime64[ns]
PostalCode           334544 non-null int64
LastSeen             334544 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(6), object(6)
memory usage: 40.8+ MB


In [13]:
data['Price'].value_counts()

0       9746
500     5317
1500    5006
1000    4266
1200    4255
        ... 
193        1
1112       1
5210       1
1368       1
8188       1
Name: Price, Length: 3688, dtype: int64

Очень много объявлений с ценой 0. Быть может, это очень старые машины, либо битые.

In [14]:
data[data['Price'] == 0]['Kilometer'].value_counts()

150000    6923
5000       945
125000     619
100000     399
90000      156
80000      137
20000      125
30000       94
70000       91
50000       76
60000       70
10000       59
40000       52
Name: Kilometer, dtype: int64

Да, так и есть - абсолютное большинство с пробегом 150000. Оставлю эти 0 без изменений.

In [15]:
data.reset_index(drop=True, inplace=True)

Составил список категориальных данных:

In [16]:
data_ordinal = data[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']]

## Вывод

В данном этапе я избавился от пропусков в столбцах `VehicleType`, `Gearbox`, `Model`, `FuelType`, `NotRepaired`.

Поменял типы данных столбцов `DateCrawled`, `DateCreated`, `LastSeen`.

Удалил выбросы, также использовал ordinal encoder для кодирования.

По итогу я удалил всего 5.6% значений.

## Обучение моделей

Обучите разные модели. Для каждой попробуйте различные гиперпараметры.

Для начала разобью данные на target и features:

In [17]:
features = data.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'Price'], axis=1)
target = data['Price']

Далее надо разбить выборку на обучающую и валидационную:

In [18]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Теперь обучу модель `CatBoostRegressor` и подберу наилучшие параметры с помощью `grid_search`:

In [29]:
cat_features = data_ordinal
model_CatBoost = CatBoostRegressor(loss_function='RMSE', random_seed=12345, cat_features=cat_features)

grid = {'learning_rate': [0.03, 0.1],
        'depth': [9, 10],
        'iterations' : [1, 200],
        'l2_leaf_reg': [1, 2]}

grid_search_result = model_CatBoost.grid_search(grid, 
                                       X=features_train, 
                                       y=target_train,
                                       verbose = True)

Наилучшее `RMSE` при значениях гиперпараметров:

In [20]:
grid_search_result['params']

Меняю тип данных на `category`:

In [21]:
features_train_lgbm = features_train.astype({
    'VehicleType' : 'category',
    'Gearbox' : 'category',
    'Model' : 'category',
    'FuelType' : 'category',
    'Brand' : 'category',
    'NotRepaired'  : 'category'
})
features_test_lgbm = features_test.astype({
    'VehicleType' : 'category',
    'Gearbox' : 'category',
    'Model' : 'category',
    'FuelType' : 'category',
    'Brand' : 'category',
    'NotRepaired'  : 'category'
})

Перебираю гиперпараметры для `LGBMRegressor`:

In [22]:
gridParams = {
    'learning_rate': [1],
    'max_depth' : [1, 100],
    'num_leaves' : [2, 98],
    'n_estimators' : [1, 100]
    }

In [23]:
model_lgbm = lgb.LGBMRegressor(metric='rmse',random_state=12345)
grid = GridSearchCV(model_lgbm, gridParams, cv=2)
grid.fit(features_train_lgbm, target_train)
print(grid.best_params_)

{'learning_rate': 1, 'max_depth': 100, 'n_estimators': 100, 'num_leaves': 98}


Для обучения дерева решений кодирую категориальные признаки с помощью `OrdinalEncoder`:

In [24]:
data_tree = data
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(data_ordinal), columns=data_ordinal.columns)
data_tree[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']] = data_ordinal[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,3.0,1993,1.0,0,116.0,150000,0,6.0,38.0,0.0,2016-03-24,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,2.0,2011,1.0,190,166.0,125000,5,2.0,1.0,1.0,2016-03-24,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,6.0,2004,0.0,163,117.0,125000,8,2.0,14.0,0.0,2016-03-14,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,5.0,2001,1.0,75,116.0,150000,6,6.0,38.0,0.0,2016-03-17,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,5.0,2008,1.0,69,101.0,90000,7,2.0,31.0,0.0,2016-03-31,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334544 entries, 0 to 334543
Data columns (total 15 columns):
DateCrawled          334544 non-null datetime64[ns]
Price                334544 non-null int64
VehicleType          334544 non-null float64
RegistrationYear     334544 non-null int64
Gearbox              334544 non-null float64
Power                334544 non-null int64
Model                334544 non-null float64
Kilometer            334544 non-null int64
RegistrationMonth    334544 non-null int64
FuelType             334544 non-null float64
Brand                334544 non-null float64
NotRepaired          334544 non-null float64
DateCreated          334544 non-null datetime64[ns]
PostalCode           334544 non-null int64
LastSeen             334544 non-null datetime64[ns]
dtypes: datetime64[ns](3), float64(6), int64(6)
memory usage: 38.3 MB


Для дерева решений разбиваю на выборки таблицу с закодированными категориальными признаками:

In [25]:
features_tree = data_tree.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'Price'], axis=1)
target_tree = data_tree['Price']
features_train_tree, features_test_tree, target_train_tree, target_test_tree = train_test_split(
    features_tree, target_tree, test_size=0.25, random_state=12345)

Выполняю масштабирование признаков:

In [26]:
numeric = ['VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand', 'NotRepaired']

scaler = StandardScaler()
scaler.fit(features_train_tree[numeric])
features_train_tree[numeric] = scaler.transform(features_train_tree[numeric])
features_test_tree[numeric] = scaler.transform(features_test_tree[numeric])

Подбираю гиперпараметры для `DecisionTreeRegressor`:

In [27]:
best_depth = 0
max_result = 10000
for depth in range(1, 14):
    model_tree = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    model_tree.fit(features_train_tree, target_train_tree)
    predictions_valid = model_tree.predict(features_test_tree)
    result = mean_squared_error(target_test_tree, predictions_valid)**0.5
    if result < max_result:
        best_depth = depth
        max_result = result
print('max_depth:', best_depth)
print('accuracy:', max_result) 

max_depth: 13
accuracy: 2039.0947251787165


## Вывод

В данном шаге я разбил данные на выдорки и сделал масштабирование признаков.

Подбирал значения гиперпараметров для трёх моделей: `CatBoostRegressor`, `LGBMRegressor`, `DecisionTreeRegressor`, использовал для этого `grid_search`, `GridSearchCV` и `цикл for`, соответственно.

## Анализ моделей

Проанализируйте скорость работы и качество моделей.

Подставляю наилучшие параметры в модель CatBoostRegressor:

In [30]:
model_CatBoost = CatBoostRegressor(loss_function='RMSE', random_seed=12345, learning_rate=0.1, depth=10, iterations=200, l2_leaf_reg=2, cat_features=cat_features)

Узнаю время обучения и предсказания:

In [31]:
%%time
model_CatBoost.fit(features_train, target_train, verbose=100)

0:	learn: 4230.9547886	total: 1.64s	remaining: 5m 27s
100:	learn: 1718.8659367	total: 2m 34s	remaining: 2m 31s
199:	learn: 1616.6608960	total: 5m 10s	remaining: 0us
CPU times: user 4min 52s, sys: 19.6 s, total: 5min 11s
Wall time: 5min 13s


In [32]:
%%time
predict_test = model_CatBoost.predict(features_test)

CPU times: user 610 ms, sys: 29.4 ms, total: 640 ms
Wall time: 629 ms


Высчитываю RMSE:

In [33]:
print('Значение RMSE: ',mean_squared_error(target_test, predict_test)**0.5)

Значение RMSE:  1719.060160777975


Подставляю наилучшие параметры в модель LGBMRegressor:

In [34]:
model_lgbm = lgb.LGBMRegressor(random_state=12345,max_depth=100,num_leaves=98,n_estimators=100,learning_rate=1)

Узнаю время обучения и предсказания:

In [37]:
%%time
model_lgbm.fit(features_train_lgbm, target_train)

CPU times: user 6min 48s, sys: 1.75 s, total: 6min 50s
Wall time: 6min 53s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=1, max_depth=100,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=98, objective=None,
              random_state=12345, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [38]:
%%time
predictions = model_lgbm.predict(features_test_lgbm)

CPU times: user 1.07 s, sys: 3.33 ms, total: 1.07 s
Wall time: 1.07 s


Высчитываю RMSE:

In [39]:
print('Значение RMSE: ',mean_squared_error(target_test, predictions)**0.5)

Значение RMSE:  1923.761139344335


Подставляю наилучшие параметры в модель DecisionTreeRegressor:

In [40]:
model_tree = DecisionTreeRegressor(random_state=12345, max_depth=13)

Узнаю время обучения и предсказания:

In [41]:
%%time
model_tree.fit(features_train_tree, target_train_tree)

CPU times: user 1.31 s, sys: 3.03 ms, total: 1.32 s
Wall time: 1.33 s


DecisionTreeRegressor(criterion='mse', max_depth=13, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=12345, splitter='best')

In [42]:
%%time
predictions_test = model_tree.predict(features_test_tree)

CPU times: user 22.4 ms, sys: 21 µs, total: 22.5 ms
Wall time: 21 ms


Высчитываю RMSE:

In [43]:
print('Значение RMSE: ',mean_squared_error(target_test_tree, predictions_test)**0.5)

Значение RMSE:  2039.0947251787165


Объединяю данные в таблицу для наглядности:

In [49]:
columns = ['Модель','RMSE','Время обучения','Время предсказания']
data_time = [['CatBoostRegressor', 1719, '5min 13s', '629ms'],
             ['LGBMRegressor', 1923, '6min 53s','1.07s'],
             ['DecisionTreeRegressor', 2039,'1.33s','21ms']]
import pandas as pd
display(pd.DataFrame(data=data_time, columns=columns))

,Модель,RMSE,Время обучения,Время предсказания
0,CatBoostRegressor,1719,5min 13s,629ms
1,LGBMRegressor,1923,6min 53s,1.07s
2,DecisionTreeRegressor,2039,1.33s,21ms


## Вывод

По итогу лучшее RMSE дала модель CatBoostRegressor, я уверен, что подобных значений можно было добиться и от LGBMRegressor, но у него параметры перебираются больше трёх часов и это просто жесть (в масштабах учебного проекта).

Время предсказаний у всех моделей очень малое.

Хотя сложно назвать такие значения хорошими, ведь это значит, что модель ошибается на 1700 евро в обе стороны во время предсказаний. С другой стороны у нас очень мало признаков дано изначально, да и сама площадка объявлений позволяет пользователям не заполнять многие поля, в результате чего в начальном этапе я столкнулся с таким большим количеством пропусков в определённых признаках. Всё это негативно сказывается на точности предсказаний.